# Char-RNN

In [28]:
import random

from itertools import chain
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM
from keras.preprocessing.text import text_to_word_sequence, one_hot

## Args

In [30]:
GPU_ACTIVE = False

## Load

In [17]:
%store -r descs
len(descs)

171161

In [18]:
random.sample(descs, 10)

['Get the options for a plugin. DCNL IMPORTANT NOTE: This method only returns the options for a plugin DCNL that was previously configured using set_plugin_options. If you want DCNL to get the default options for a plugin, get a plugin instance and DCNL perform a plugin.get_options() DCNL :return: An OptionList with the plugin options.',
 'Merge this guy with another SimAbstractMemory instance',
 'Computes the mean squared error. DCNL This function computes the mean squared error between two lists DCNL of numbers. DCNL Parameters DCNL actual : list of numbers, numpy array DCNL The ground truth value DCNL predicted : same type as actual DCNL The predicted value DCNL Returns DCNL score : double DCNL The mean squared error between actual and predicted',
 'Get standard_stats: only for displayed processes. DCNL Stats grabbed inside this method: DCNL * nice and memory_info',
 'If a specific interface is passed into DCNL L{SSHProtocolChecker.registerChecker}, that interface should be DCNL reg

In [22]:
SEQ_LEN = max(len(desc) for desc in descs)

## Preprocessing

In [19]:
def chars_split(descs):
    chars = list(set(chain.from_iterable(desc for desc in descs)))
    char_ix = {char:ix for ix, char in enumerate(chars)}
    ix_char = {ix:char for ix, char in enumerate(chars)}
    descs = [[char_ix[char] for char in desc] for desc in descs]
    return descs, char_ix, ix_char

In [20]:
%time descs, dir_map, rev_map = chars_split(descs)

CPU times: user 1.94 s, sys: 12 ms, total: 1.95 s
Wall time: 1.96 s


## TT prepare

## Model

In [31]:
HIDDEN_DIM = 300
LSTM_CLASS = LSTM if not GPU_ACTIVE else CuDNNLSTM

In [24]:
model = Sequential()

In [ ]:
model.add(LSTM_CLASS(HIDDEN_DIM, return_sequences=True))